In [1]:
from os import getcwd
from pathlib import Path

import flopy as fp
import numpy as np
import pandas as pd

In [2]:
workspace = Path(getcwd(), "model")
workspace

WindowsPath('c:/Users/buckl/Documents/Github/flopy_mf6_work/notebooks/implementing/gwf_advtidal/model')

In [3]:
sim_name = "gwf_advtidal"
sim = fp.mf6.MFSimulation(
    sim_name=sim_name,
    version="mf6",
    exe_name=Path("C:\\Users\\buckl\\Documents\\Software\\mf6.4.1\\bin\\mf6.exe"),
    sim_ws=workspace
)
sim

sim_name = gwf_advtidal
sim_path = c:\Users\buckl\Documents\Github\flopy_mf6_work\notebooks\implementing\gwf_advtidal\model
exe_name = C:\Users\buckl\Documents\Software\mf6.4.1\bin\mf6.exe

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = gwf_advtidal



In [4]:
fp.mf6.ModflowTdis(
    simulation=sim,
    time_units="days",
    nper=4,
    perioddata=[
        (1.0, 1, 1.0),
        (10.0, 120, 1.0),
        (10.0, 120, 1.0),
        (10.0, 120, 1.0)
    ],
    filename=f"{sim_name}.tdis",
    pname="tdis"
)

package_name = tdis
filename = gwf_advtidal.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = gwf_advtidal

Block options
--------------------
time_units
{internal}
('days')


Block dimensions
--------------------
nper
{internal}
(4)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([( 1.,   1, 1.), (10., 120, 1.), (10., 120, 1.), (10., 120, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



In [5]:
fp.mf6.ModflowIms(
    simulation=sim,
    outer_dvclose=1e-9,
    outer_maximum=50,
    inner_maximum=100,
    inner_dvclose=1e-9,
    rcloserecord=[1e-6, "strict"],
    filename=f"{sim_name}.ims",
    pname="ims"
)

package_name = ims
filename = gwf_advtidal.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = gwf_advtidal

Block nonlinear
--------------------
outer_dvclose
{internal}
(1e-09)

outer_maximum
{internal}
(50)


Block linear
--------------------
inner_maximum
{internal}
(100)

inner_dvclose
{internal}
(1e-09)

rcloserecord
{internal}
(rec.array([('inner_rclose', 1.e-06, 'strict')],
          dtype=[('inner_rclose_label', 'O'), ('inner_rclose', '<f8'), ('rclose_option', 'O')]))



In [6]:
model = fp.mf6.ModflowGwf(
    simulation=sim,
    modelname=sim_name,
    model_nam_file=f"{sim_name}.nam",
    save_flows=True
)
model

name = gwf_advtidal
model_type = gwf6
version = mf6
model_relative_path = .


In [7]:
fp.mf6.ModflowGwfdis(
    model=model,
    length_units="meters",
    nlay=3,
    nrow=15,
    ncol=10,
    delr=500.0,
    delc=500.0,
    top=50.0,
    botm=[
        5.0,
        -10.0,
        -100.0
    ],
    filename=f"{sim_name}.dis",
    pname="dis"
)

package_name = dis
filename = gwf_advtidal.dis
package_type = dis
model_or_simulation_package = model
model_name = gwf_advtidal

Block options
--------------------
length_units
{internal}
('meters')


Block dimensions
--------------------
nlay
{internal}
(3)

nrow
{internal}
(15)

ncol
{internal}
(10)


Block griddata
--------------------
delr
{constant 500.0}

delc
{constant 500.0}

top
{constant 50.0}

botm
Layer_1{constant 5.0}
Layer_2{constant -10.0}
Layer_3{constant -100.0}



In [8]:
fp.mf6.ModflowGwfnpf(
    model=model,
    cvoptions=[(True, "DEWATERED")],
    perched=True,
    save_specific_discharge=True,
    icelltype=[1, 0, 0],
    k=[
        5.0,
        0.1,
        4.0
    ],
    k33=[
        0.5,
        0.005,
        0.1
    ],
    filename=f"{sim_name}.npf",
    pname="npf"
)

package_name = npf
filename = gwf_advtidal.npf
package_type = npf
model_or_simulation_package = model
model_name = gwf_advtidal

Block options
--------------------
cvoptions
{internal}
([(True, 'DEWATERED')])

perched
{internal}
(True)

save_specific_discharge
{internal}
(True)


Block griddata
--------------------
icelltype
Layer_1{constant 1}
Layer_2{constant 0}
Layer_3{constant 0}

k
Layer_1{constant 5.0}
Layer_2{constant 0.1}
Layer_3{constant 4.0}

k33
Layer_1{constant 0.5}
Layer_2{constant 0.005}
Layer_3{constant 0.1}



In [9]:
fp.mf6.ModflowGwfic(
    model=model,
    strt=50.0,
    filename=f"{sim_name}.ic",
    pname="ic"
)

package_name = ic
filename = gwf_advtidal.ic
package_type = ic
model_or_simulation_package = model
model_name = gwf_advtidal

Block griddata
--------------------
strt
{constant 50.0}



In [10]:
fp.mf6.ModflowGwfsto(
    model=model,
    iconvert=1,
    ss=1e-6,
    sy=0.2,
    steady_state=True,
    transient=True,
    filename=f"{sim_name}.sto",
    pname="sto"
)

package_name = sto
filename = gwf_advtidal.sto
package_type = sto
model_or_simulation_package = model
model_name = gwf_advtidal

Block griddata
--------------------
iconvert
{constant 1}

ss
{constant 1e-06}

sy
{constant 0.2}


Block period
--------------------
steady-state
{internal}
(True)

transient
{internal}
(True)



In [11]:
ghb_ts_df = pd.DataFrame([
    ( 0.   , -2.5), ( 0.05 , -2.1), ( 0.206,  2.1), ( 0.256,  2.5),
    ( 0.306,  2. ), ( 0.462, -2.4), ( 0.512, -2.8), ( 0.562, -2.4),
    ( 0.718,  2.4), ( 0.768,  2.8), ( 0.818,  2.4), ( 0.974, -2.1),
    ( 1.024, -2.5), ( 1.074, -2.1), ( 1.23 ,  2.1), ( 1.28 ,  2.5),
    ( 1.33 ,  2. ), ( 1.486, -2.4), ( 1.536, -2.8), ( 1.586, -2.4),
    ( 1.742,  2.4), ( 1.792,  2.8), ( 1.842,  2.4), ( 1.998, -2.1),
    ( 2.048, -2.5), ( 2.098, -2.1), ( 2.254,  2.1), ( 2.304,  2.5),
    ( 2.354,  2. ), ( 2.51 , -2.4), ( 2.56 , -2.8), ( 2.61 , -2.4),
    ( 2.766,  2.4), ( 2.816,  2.8), ( 2.866,  2.4), ( 3.022, -2.1),
    ( 3.072, -2.5), ( 3.122, -2.1), ( 3.278,  2.1), ( 3.328,  2.5),
    ( 3.378,  2. ), ( 3.534, -2.4), ( 3.584, -2.8), ( 3.634, -2.4),
    ( 3.79 ,  2.4), ( 3.84 ,  2.8), ( 3.89 ,  2.4), ( 4.046, -2.1),
    ( 4.096, -2.5), ( 4.146, -2.1), ( 4.302,  2.1), ( 4.352,  2.5),
    ( 4.402,  2. ), ( 4.558, -2.4), ( 4.608, -2.8), ( 4.658, -2.4),
    ( 4.814,  2.4), ( 4.864,  2.8), ( 4.914,  2.4), ( 5.07 , -2.1),
    ( 5.12 , -2.5), ( 5.17 , -2.1), ( 5.326,  2.1), ( 5.376,  2.5),
    ( 5.426,  2. ), ( 5.582, -2.4), ( 5.632, -2.8), ( 5.682, -2.4),
    ( 5.838,  2.4), ( 5.888,  2.8), ( 5.938,  2.4), ( 6.094, -2.1),
    ( 6.144, -2.5), ( 6.194, -2.1), ( 6.35 ,  2.1), ( 6.4  ,  2.5),
    ( 6.45 ,  2. ), ( 6.606, -2.4), ( 6.656, -2.8), ( 6.706, -2.4),
    ( 6.862,  2.4), ( 6.912,  2.8), ( 6.962,  2.4), ( 7.118, -2.1),
    ( 7.168, -2.5), ( 7.218, -2.1), ( 7.374,  2.1), ( 7.424,  2.5),
    ( 7.474,  2. ), ( 7.63 , -2.4), ( 7.68 , -2.8), ( 7.73 , -2.4),
    ( 7.886,  2.4), ( 7.936,  2.8), ( 7.986,  2.4), ( 8.142, -2.1),
    ( 8.192, -2.5), ( 8.242, -2.1), ( 8.398,  2.1), ( 8.448,  2.5),
    ( 8.498,  2. ), ( 8.654, -2.4), ( 8.704, -2.8), ( 8.754, -2.4),
    ( 8.91 ,  2.4), ( 8.96 ,  2.8), ( 9.01 ,  2.4), ( 9.166, -2.1),
    ( 9.216, -2.5), ( 9.266, -2.1), ( 9.422,  2.1), ( 9.472,  2.5),
    ( 9.522,  2. ), ( 9.678, -2.4), ( 9.728, -2.8), ( 9.778, -2.4),
    ( 9.934,  2.4), ( 9.984,  2.8), (10.034,  2.4), (10.19 , -2.1),
    (10.24 , -2.5), (10.29 , -2.1), (10.446,  2.1), (10.496,  2.5),
    (10.546,  2. ), (10.702, -2.4), (10.752, -2.8), (10.802, -2.4),
    (10.958,  2.4), (11.008,  2.8), (11.058,  2.4), (11.214, -2.1),
    (11.264, -2.5), (11.314, -2.1), (11.47 ,  2.1), (11.52 ,  2.5),
    (11.57 ,  2. ), (11.726, -2.4), (11.776, -2.8), (11.826, -2.4),
    (11.982,  2.4), (12.032,  2.8), (12.082,  2.4), (12.238, -2.1),
    (12.288, -2.5), (12.338, -2.1), (12.494,  2.1), (12.544,  2.5),
    (12.594,  2. ), (12.75 , -2.4), (12.8  ,  2.8), (12.85 , -2.4),
    (13.006,  2.4), (13.056,  2.8), (13.106,  2.4), (13.262, -2.1),
    (13.312, -2.5), (13.362, -2.1), (13.518,  2.1), (13.568,  2.5),
    (13.618,  2. ), (13.774, -2.4), (13.824, -2.8), (13.874, -2.4),
    (14.03 ,  2.4), (14.08 ,  2.8), (14.13 ,  2.4), (14.286, -2.1),
    (14.336, -2.5), (14.386, -2.1), (14.542,  2.1), (14.592,  2.5),
    (14.642,  2. ), (14.798, -2.4), (14.848, -2.8), (14.898, -2.4),
    (15.054,  2.4), (15.104,  2.8), (15.154,  2.4), (15.31 , -2.1),
    (15.36 , -2.5), (15.41 , -2.1), (15.566,  2.1), (15.616,  2.5),
    (15.666,  2. ), (15.822, -2.4), (15.872, -2.8), (15.922, -2.4),
    (16.078,  2.4), (16.128,  2.8), (16.178,  2.4), (16.334, -2.1),
    (16.384, -2.5), (16.434, -2.1), (16.59 ,  2.1), (16.64 ,  2.5),
    (16.69 ,  2. ), (16.846, -2.4), (16.896, -2.8), (16.946, -2.4),
    (17.102,  2.4), (17.152,  2.8), (17.202,  2.4), (17.358, -2.1),
    (17.408, -2.5), (17.458, -2.1), (17.614,  2.1), (17.664,  2.5),
    (17.714,  2. ), (17.87 , -2.4), (17.92 , -2.8), (17.97 , -2.4),
    (18.126,  2.4), (18.176,  2.8), (18.226,  2.4), (18.382, -2.1),
    (18.432, -2.5), (18.482, -2.1), (18.638,  2.1), (18.688,  2.5),
    (18.738,  2. ), (18.894, -2.4), (18.944, -2.8), (18.994, -2.4),
    (19.15 ,  2.4), (19.2  ,  2.8), (19.25 ,  2.4), (19.406, -2.1),
    (19.456, -2.5), (19.506, -2.1), (19.662,  2.1), (19.712,  2.5),
    (19.762,  2. ), (19.918, -2.4), (19.968, -2.8), (20.018, -2.4),
    (20.174,  2.4), (20.224,  2.8), (20.274,  2.4), (20.43 , -2.1),
    (20.48 , -2.5), (20.53 , -2.1), (20.686,  2.1), (20.736,  2.5),
    (20.786,  2. ), (20.942, -2.4), (20.992, -2.8), (21.042, -2.4),
    (21.198,  2.4), (21.248,  2.8), (21.298,  2.4), (21.454, -2.1),
    (21.504, -2.5), (21.554, -2.1), (21.71 ,  2.1), (21.76 ,  2.5),
    (21.81 ,  2. ), (21.966, -2.4), (22.016, -2.8), (22.066, -2.4),
    (22.222,  2.4), (22.272,  2.8), (22.322,  2.4), (22.478, -2.1),
    (22.528, -2.5), (22.578, -2.1), (22.734,  2.1), (22.784,  2.5),
    (22.834,  2. ), (22.99 , -2.4), (23.04 , -2.8), (23.09 , -2.4),
    (23.246,  2.4), (23.296,  2.8), (23.346,  2.4), (23.502, -2.1),
    (23.552, -2.5), (23.602, -2.1), (23.758,  2.1), (23.808,  2.5),
    (23.858,  2. ), (24.014, -2.4), (24.064, -2.8), (24.114, -2.4),
    (24.27 ,  2.4), (24.32 ,  2.8), (24.37 ,  2.4), (24.526, -2.1),
    (24.576, -2.5), (24.626, -2.1), (24.782,  2.1), (24.832,  2.5),
    (24.882,  2. ), (25.038, -2.4), (25.088, -2.8), (25.138, -2.4),
    (25.294,  2.4), (25.344,  2.8), (25.394,  2.4), (25.55 , -2.1),
    (25.6  ,  2.5), (25.65 , -2.1), (25.806,  2.1), (25.856,  2.5),
    (25.906,  2. ), (26.062, -2.4), (26.112, -2.8), (26.162, -2.4),
    (26.318,  2.4), (26.368,  2.8), (26.418,  2.4), (26.574, -2.1),
    (26.624, -2.5), (26.674, -2.1), (26.83 ,  2.1), (26.88 ,  2.5),
    (26.93 ,  2. ), (27.086, -2.4), (27.136, -2.8), (27.186, -2.4),
    (27.342,  2.4), (27.392,  2.8), (27.442,  2.4), (27.598, -2.1),
    (27.648, -2.5), (27.698, -2.1), (27.854,  2.1), (27.904,  2.5),
    (27.954,  2. ), (28.11 , -2.4), (28.16 , -2.8), (28.21 , -2.4),
    (28.366,  2.4), (28.416,  2.8), (28.466,  2.4), (28.622, -2.1),
    (28.672, -2.5), (28.722, -2.1), (28.878,  2.1), (28.928,  2.5),
    (28.978,  2. ), (29.134, -2.4), (29.184, -2.8), (29.234, -2.4),
    (29.39 ,  2.4), (29.44 ,  2.8), (29.49 ,  2.4), (29.646, -2.1),
    (29.696, -2.5), (29.746, -2.1), (29.902,  2.1), (29.952,  2.5),
    (30.002,  2. ), (30.158, -2.4), (30.208, -2.8), (30.258, -2.4),
    (30.414,  2.4), (30.464,  2.8), (30.514,  2.4), (30.67 , -2.1),
    (30.72 , -2.5), (30.77 , -2.1), (30.926,  2.1), (30.976,  2.5),
    (31.026,  2. )
], columns=["ts_time", "tides"])

def make_ghb_stress_data():
    cond_dict = {
        1: 15.0,
        2: 1500.0
    }
    for layer in [1, 2]:
        for row in range(15):
            yield ((layer, row, 9), "tides", cond_dict[layer], f"estuary-l{layer + 1}")

ghb = fp.mf6.ModflowGwfghb(
    model=model,
    boundnames=True,
    # timeseries={
    #     "time_series_namerecord": ghb_ts_df.columns[1],
    #     "interpolation_methodrecord": ["linear"],
    #     "timeseries": ghb_ts_df.to_records(index=False),
    #     "filename": f"{sim_name}.ghb.ts"
    # },
    observations={
        f"{sim_name}.ghb.obs.csv": [
            ("ghb_2_6_10", "ghb", (1, 5, 9), None),
            ("ghb_3_6_10", "ghb", (2, 5, 9), None),
            ("estuary2", "ghb", "estuary-l2", None),
            ("estuary3", "ghb", "estuary-l3", None),
        ]
    },
    maxbound=30,
    stress_period_data=list(make_ghb_stress_data()),
    filename=f"{sim_name}.ghb",
    pname="ghb-tidal"
)
fp.mf6.ModflowUtlts(
    parent_package=ghb,
    time_series_namerecord=ghb_ts_df.columns[1:].tolist(),
    interpolation_methodrecord=["linear"],
    timeseries=ghb_ts_df.to_records(index=False),
    filename=f"{sim_name}.ghb.ts",
    pname="ghb_ts"
)

package_name = ghb_ts
filename = gwf_advtidal.ghb.ts
package_type = ts
model_or_simulation_package = model
model_name = gwf_advtidal
parent_file = ghb-tidal

Block attributes
--------------------
time_series_namerecord
{internal}
(rec.array([('tides',)],
          dtype=[('time_series_names', 'O')]))

interpolation_methodrecord
{internal}
(rec.array([('linear',)],
          dtype=[('interpolation_method', 'O')]))


Block timeseries
--------------------
timeseries
{internal}
(rec.array([( 0.   , -2.5), ( 0.05 , -2.1), ( 0.206,  2.1), ( 0.256,  2.5),
           ( 0.306,  2. ), ( 0.462, -2.4), ( 0.512, -2.8), ( 0.562, -2.4),
           ( 0.718,  2.4), ( 0.768,  2.8), ( 0.818,  2.4), ( 0.974, -2.1),
           ( 1.024, -2.5), ( 1.074, -2.1), ( 1.23 ,  2.1), ( 1.28 ,  2.5),
           ( 1.33 ,  2. ), ( 1.486, -2.4), ( 1.536, -2.8), ( 1.586, -2.4),
           ( 1.742,  2.4), ( 1.792,  2.8), ( 1.842,  2.4), ( 1.998, -2.1),
           ( 2.048, -2.5), ( 2.098, -2.1), ( 2.254,  2.1), ( 2.304,  2

In [12]:
model.package_dict['ghb-tidal']

package_name = ghb-tidal
filename = gwf_advtidal.ghb
package_type = ghb
model_or_simulation_package = model
model_name = gwf_advtidal

Block options
--------------------
boundnames
{internal}
(True)

ts_filerecord
{internal}
(rec.array([('gwf_advtidal.ghb.ts',)],
          dtype=[('ts6_filename', 'O')]))

obs_filerecord
{internal}
(rec.array([('gwf_advtidal.ghb.obs',)],
          dtype=[('obs6_filename', 'O')]))


Block dimensions
--------------------
maxbound
{internal}
(30)


Block period
--------------------
stress_period_data
{0: <flopy.mf6.data.mfdataplist.PandasListStorage object at 0x0000020E20FC4210>}


In [13]:
model.package_dict['ghb-tidal'].stress_period_data.get_data(key=0)

rec.array([((1, 0, 9), 'tides',   15., 'estuary-l2'),
           ((1, 1, 9), 'tides',   15., 'estuary-l2'),
           ((1, 2, 9), 'tides',   15., 'estuary-l2'),
           ((1, 3, 9), 'tides',   15., 'estuary-l2'),
           ((1, 4, 9), 'tides',   15., 'estuary-l2'),
           ((1, 5, 9), 'tides',   15., 'estuary-l2'),
           ((1, 6, 9), 'tides',   15., 'estuary-l2'),
           ((1, 7, 9), 'tides',   15., 'estuary-l2'),
           ((1, 8, 9), 'tides',   15., 'estuary-l2'),
           ((1, 9, 9), 'tides',   15., 'estuary-l2'),
           ((1, 10, 9), 'tides',   15., 'estuary-l2'),
           ((1, 11, 9), 'tides',   15., 'estuary-l2'),
           ((1, 12, 9), 'tides',   15., 'estuary-l2'),
           ((1, 13, 9), 'tides',   15., 'estuary-l2'),
           ((1, 14, 9), 'tides',   15., 'estuary-l2'),
           ((2, 0, 9), 'tides', 1500., 'estuary-l3'),
           ((2, 1, 9), 'tides', 1500., 'estuary-l3'),
           ((2, 2, 9), 'tides', 1500., 'estuary-l3'),
           ((2, 3, 9), 

In [14]:
wel_ts_df = pd.DataFrame([
    (0.0, 0.0, 0.0, 0.0),
    (1.0, -200.0, 0.0, -100.0),
    (11.0, -1800.0, -500.0, -200.0),
    (21.0, -200.0, -400.0, -300.0),
    (31.0, 0.0, -600.0, -400.0)
], columns=["ts_time", "well_1_rate", "well_2_rate", "well_6_rate"])

wel = fp.mf6.ModflowGwfwel(
    model=model,
    boundnames=True,
    timeseries={
        "time_series_namerecord": wel_ts_df.columns[1:].tolist(),
        "interpolation_methodrecord": ["stepwise", "stepwise", "stepwise"],
        "timeseries": wel_ts_df.to_records(index=False),
        "filename": f"{sim_name}.wel.ts"
    },
    maxbound=5,
    stress_period_data={
        1: [
            ((0, 11, 2), -50.0, None),
            ((2, 4, 7), "well_1_rate", "well_1"),
            ((2, 3, 2), "well_2_rate", "well_2")
        ],
        2: [
            ((2, 4, 7), "well_1_rate", "well_1"),
            ((2, 3, 2), "well_2_rate", "well_2"),
            ((0, 11, 2), -10.0, None),
            ((0, 2, 4), -20.0, None),
            ((0, 13, 5), -40.0, None)
        ],
        3: [
            ((2, 4, 7), "well_1_rate", "well_1"),
            ((2, 3, 2), "well_2_rate", "well_2"),
            ((0, 11, 2), -10.0, None),
            ((0, 2, 4), -20.0, None),
            ((0, 13, 5), -40.0, None)
        ]
    },
    filename=f"{sim_name}.wel",
    pname="wel"
)
fp.mf6.ModflowUtlts(
    parent_package=wel,
    time_series_namerecord=wel_ts_df.columns[1:].tolist(),
    interpolation_methodrecord=["stepwise", "stepwise", "stepwise"],
    timeseries=wel_ts_df.to_records(index=False),
    filename=f"{sim_name}.wel.ts",
    pname="wel_ts"
)

package_name = wel_ts
filename = gwf_advtidal.wel.ts
package_type = ts
model_or_simulation_package = model
model_name = gwf_advtidal
parent_file = wel

Block attributes
--------------------
time_series_namerecord
{internal}
(rec.array([('well_1_rate', 'well_2_rate', 'well_6_rate')],
          dtype=[('time_series_names_0', 'O'), ('time_series_names_1', 'O'), ('time_series_names_2', 'O')]))

interpolation_methodrecord
{internal}
(rec.array([('stepwise', 'stepwise', 'stepwise')],
          dtype=[('interpolation_method_0', 'O'), ('interpolation_method_1', 'O'), ('interpolation_method_2', 'O')]))


Block timeseries
--------------------
timeseries
{internal}
(rec.array([( 0.,     0.,    0.,    0.), ( 1.,  -200.,    0., -100.),
           (11., -1800., -500., -200.), (21.,  -200., -400., -300.),
           (31.,     0., -600., -400.)],
          dtype=[('ts_time', '<f8'), ('well_1_rate', '<f8'), ('well_2_rate', '<f8'), ('well_6_rate', '<f8')]))



In [15]:
model.package_dict['wel'].stress_period_data.get_data(key=1)

rec.array([((0, 11, 2), -50.0, None),
           ((2, 4, 7), 'well_1_rate', 'well_1'),
           ((2, 3, 2), 'well_2_rate', 'well_2')],
          dtype=[('cellid', 'O'), ('q', 'O'), ('boundname', 'O')])

In [16]:
model.package_dict['wel'].stress_period_data.get_data(key=2)

rec.array([((2, 4, 7), 'well_1_rate', 'well_1'),
           ((2, 3, 2), 'well_2_rate', 'well_2'),
           ((0, 11, 2), -10.0, None), ((0, 2, 4), -20.0, None),
           ((0, 13, 5), -40.0, None)],
          dtype=[('cellid', 'O'), ('q', 'O'), ('boundname', 'O')])

In [17]:
model.package_dict['wel'].stress_period_data.get_data(key=3)

rec.array([((2, 4, 7), 'well_1_rate', 'well_1'),
           ((2, 3, 2), 'well_2_rate', 'well_2'),
           ((0, 11, 2), -10.0, None), ((0, 2, 4), -20.0, None),
           ((0, 13, 5), -40.0, None)],
          dtype=[('cellid', 'O'), ('q', 'O'), ('boundname', 'O')])

In [18]:
riv_ts_df = pd.DataFrame([
    ( 0., 40., 41. ), ( 1., 41., 41.5), ( 2., 43., 42. ),
    ( 3., 45., 42.8), ( 4., 44., 43. ), ( 6., 43., 43.1),
    ( 9., 42., 42.4), (11., 41., 41.5), (31., 40., 41. )
], columns=["ts_time", "river_stage_1", "river_stage_2"])

riv = fp.mf6.ModflowGwfriv(
    model=model,
    boundnames=True,
    maxbound=20,
    stress_period_data=[
        ((0, 2, 0), 'river_stage_1', 1001.0, 35.9, None),
        ((0, 3, 1), 'river_stage_1', 1002.0, 35.8, None),
        ((0, 4, 2), 'river_stage_1', 1003.0, 35.7, None),
        ((0, 4, 3), 'river_stage_1', 1004.0, 35.6, None),
        ((0, 5, 4), 'river_stage_1', 1005.0, 35.5, None),
        ((0, 5, 5), 'river_stage_1', 1006.0, 35.4, 'riv1_c6'),
        ((0, 5, 6), 'river_stage_1', 1007.0, 35.3, 'riv1_c7'),
        ((0, 4, 7), 'river_stage_1', 1008.0, 35.2, None),
        ((0, 4, 8), 'river_stage_1', 1009.0, 35.1, None),
        ((0, 4, 9), 'river_stage_1', 1010.0, 35.0, None),
        ((0, 9, 0), 'river_stage_2', 1001.0, 36.9, 'riv2_upper'),
        ((0, 8, 1), 'river_stage_2', 1002.0, 36.8, 'riv2_upper'),
        ((0, 7, 2), 'river_stage_2', 1003.0, 36.7, 'riv2_upper'),
        ((0, 6, 3), 'river_stage_2', 1004.0, 36.6, None),
        ((0, 6, 4), 'river_stage_2', 1005.0, 36.5, None),
        ((0, 5, 5), 'river_stage_2', 1006.0, 36.4, 'riv2_c6'),
        ((0, 5, 6), 'river_stage_2', 1007.0, 36.3, 'riv2_c7'),
        ((0, 6, 7), 'river_stage_2', 1008.0, 36.2, None),
        ((0, 6, 8), 'river_stage_2', 1009.0, 36.1, None),
        ((0, 6, 9), 'river_stage_2', 1010.0, 36.0, None)
    ],
    filename=f"{sim_name}.riv",
    pname="riv"
)
fp.mf6.ModflowUtlts(
    parent_package=riv,
    time_series_namerecord=riv_ts_df.columns[1:].tolist(),
    interpolation_methodrecord=["linear", "stepwise"],
    timeseries=riv_ts_df.to_records(index=False),
    filename=f"{sim_name}.riv.ts",
    pname="riv_ts"
)

package_name = riv_ts
filename = gwf_advtidal.riv.ts
package_type = ts
model_or_simulation_package = model
model_name = gwf_advtidal
parent_file = riv

Block attributes
--------------------
time_series_namerecord
{internal}
(rec.array([('river_stage_1', 'river_stage_2')],
          dtype=[('time_series_names_0', 'O'), ('time_series_names_1', 'O')]))

interpolation_methodrecord
{internal}
(rec.array([('linear', 'stepwise')],
          dtype=[('interpolation_method_0', 'O'), ('interpolation_method_1', 'O')]))


Block timeseries
--------------------
timeseries
{internal}
(rec.array([( 0., 40., 41. ), ( 1., 41., 41.5), ( 2., 43., 42. ),
           ( 3., 45., 42.8), ( 4., 44., 43. ), ( 6., 43., 43.1),
           ( 9., 42., 42.4), (11., 41., 41.5), (31., 40., 41. )],
          dtype=[('ts_time', '<f8'), ('river_stage_1', '<f8'), ('river_stage_2', '<f8')]))



In [19]:
model.package_dict['riv'].stress_period_data.get_data(key=0)

rec.array([((0, 2, 0), 'river_stage_1', 1001., 35.9, None),
           ((0, 3, 1), 'river_stage_1', 1002., 35.8, None),
           ((0, 4, 2), 'river_stage_1', 1003., 35.7, None),
           ((0, 4, 3), 'river_stage_1', 1004., 35.6, None),
           ((0, 5, 4), 'river_stage_1', 1005., 35.5, None),
           ((0, 5, 5), 'river_stage_1', 1006., 35.4, 'riv1_c6'),
           ((0, 5, 6), 'river_stage_1', 1007., 35.3, 'riv1_c7'),
           ((0, 4, 7), 'river_stage_1', 1008., 35.2, None),
           ((0, 4, 8), 'river_stage_1', 1009., 35.1, None),
           ((0, 4, 9), 'river_stage_1', 1010., 35. , None),
           ((0, 9, 0), 'river_stage_2', 1001., 36.9, 'riv2_upper'),
           ((0, 8, 1), 'river_stage_2', 1002., 36.8, 'riv2_upper'),
           ((0, 7, 2), 'river_stage_2', 1003., 36.7, 'riv2_upper'),
           ((0, 6, 3), 'river_stage_2', 1004., 36.6, None),
           ((0, 6, 4), 'river_stage_2', 1005., 36.5, None),
           ((0, 5, 5), 'river_stage_2', 1006., 36.4, 'riv2_c6'),
 

In [20]:
rch_ts_df = pd.DataFrame([
    (0.0, 0.0015, 0.0016, 0.0017),
    (1.0, 0.001, 0.0018, 0.002),
    (11.0, 0.0015, 0.0019, 0.0017),
    (21.0, 0.0025, 0.0016, 0.0018),
    (31.0, 0.0015, 0.0018, 0.002)
], columns=["ts_time", "rch_1", "rch_2", "rch_3"])

def make_rch_stress_data(zone: int):
    overlap = {
        (0, 2): [1, 2],
        (1, 3): [1, 2],
        (2, 4): [1, 2],
        (3, 5): [1, 2],
        (3, 6): [2, 3],
        (2, 7): [2, 3],
        (1, 8): [2, 3],
        (0, 9): [2, 3]
    }
    # for cells that don't overlap
    zone_defs = [
        # zone 1
        {
            0: range(2),
            1: range(3),
            2: range(4),
            3: range(5),
            **{row: range(6) for row in range(4, 15)}
        },
        # zone 2
        {
            0: range(3, 9),
            1: range(4, 8),
            2: range(5, 7)
        },
        # zone 3
        {
            1: range(9, 10),
            2: range(8, 10),
            3: range(7, 10),
            **{row: range(6, 10) for row in range(4, 15)}
        }
    ]
    for row in range(15):
        for col in range(10):
            if (row, col) in overlap and zone in overlap[(row, col)]:
                # two zones overlap here
                yield ((0, row, col), f"rch_{zone}", 0.5, None)
            elif row in zone_defs[zone - 1] and col in zone_defs[zone - 1][row]:
                # this cell belongs to one zone
                yield ((0, row, col), f"rch_{zone}", 1.0, None)

for zone in [1, 2, 3]:
    zone_data = list(make_rch_stress_data(zone=zone))
    rch = fp.mf6.ModflowGwfrch(
        model=model,
        fixed_cell=True,
        auxiliary=["auxiliary", "multiplier"],
        auxmultname="multiplier",
        boundnames=True,
        print_input=True,
        print_flows=True,
        save_flows=True,
        maxbound=len(zone_data),
        stress_period_data=zone_data,
        filename=f"{sim_name}.rch{zone}",
        pname=f"rch-zone_{zone}"
    )
    fp.mf6.ModflowUtlts(
        parent_package=rch,
        time_series_namerecord=rch_ts_df.columns[zone],
        interpolation_methodrecord="stepwise",
        timeseries=rch_ts_df[["ts_time", rch_ts_df.columns[zone]]].to_records(index=False),
        filename=f"{sim_name}.rch{zone}.ts",
        pname=f"rch_{zone}_ts"
    )
model.package_names

['dis',
 'npf',
 'ic',
 'sto',
 'ghb-tidal',
 'obs_0',
 'ghb_ts',
 'wel',
 'wel_ts',
 'riv',
 'riv_ts',
 'rch-zone_1',
 'rch_1_ts',
 'rch-zone_2',
 'rch_2_ts',
 'rch-zone_3',
 'rch_3_ts']

In [21]:
def make_evt_stress_data():
    for row in range(15):
        for col in range(10):
            yield ((0, row, col), 50.0, 0.0004, 10.0, 0.2, 0.5, 0.3, 0.1, None)

fp.mf6.ModflowGwfevt(
    model=model,
    maxbound=150,
    nseg=3,
    stress_period_data=list(make_evt_stress_data()),
    filename=f"{sim_name}.evt",
    pname="evt"
)

package_name = evt
filename = gwf_advtidal.evt
package_type = evt
model_or_simulation_package = model
model_name = gwf_advtidal

Block dimensions
--------------------
maxbound
{internal}
(150)

nseg
{internal}
(3)


Block period
--------------------
stress_period_data
{internal}
(rec.array([((0, 0, 0), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 1), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 2), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 3), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 4), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 5), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 6), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 7), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 8), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 0, 9), 50., 0.0004, 10., 0.2, 0.5, 0.3, 0.1, None),
           ((0, 1, 0), 50., 0.0004, 10.

In [22]:
fp.mf6.ModflowGwfoc(
    model=model,
    budget_filerecord=[f"{sim_name}.cbc"],
    head_filerecord=[f"{sim_name}.hds"],
    saverecord=[
        ("head", "all"),
        ("budget", "all")
    ],
    filename=f"{sim_name}.oc",
    pname="oc"
)

package_name = oc
filename = gwf_advtidal.oc
package_type = oc
model_or_simulation_package = model
model_name = gwf_advtidal

Block options
--------------------
budget_filerecord
{internal}
(rec.array([('gwf_advtidal.cbc',)],
          dtype=[('budgetfile', 'O')]))

head_filerecord
{internal}
(rec.array([('gwf_advtidal.hds',)],
          dtype=[('headfile', 'O')]))


Block period
--------------------
saverecord
{internal}
(rec.array([('head', 'all', None), ('budget', 'all', None)],
          dtype=[('rtype', 'O'), ('ocsetting', 'O'), ('ocsetting_data', 'O')]))

printrecord
None


In [23]:
fp.mf6.ModflowUtlobs(
    parent_model_or_package=model,
    continuous={
        f"{sim_name}.obs.head.csv": [
            ("h3_13_8", "head", (2, 12, 7), None)
            # ("icf1", "flow-ja-face", (0, 4, 5), (0, 5, 5))
        ],
        f"{sim_name}.obs.flow.csv": [
            # ("h3_13_8", "head", (2, 12, 7), None),
            ("icf1", "flow-ja-face", (0, 4, 5), (0, 5, 5))
        ]
    },
    filename=f"{sim_name}.obs",
    pname="obs"
)

package_name = obs
filename = gwf_advtidal.obs
package_type = obs
model_or_simulation_package = model
model_name = gwf_advtidal

Block continuous
--------------------
continuous
{internal}
(rec.array([('icf1', 'flow-ja-face', (0, 4, 5), (0, 5, 5))],
          dtype=[('obsname', 'O'), ('obstype', 'O'), ('id', 'O'), ('id2', 'O')]))



In [24]:
model.package_names

['dis',
 'npf',
 'ic',
 'sto',
 'ghb-tidal',
 'obs_0',
 'ghb_ts',
 'wel',
 'wel_ts',
 'riv',
 'riv_ts',
 'rch-zone_1',
 'rch_1_ts',
 'rch-zone_2',
 'rch_2_ts',
 'rch-zone_3',
 'rch_3_ts',
 'evt',
 'oc',
 'obs']

In [25]:
sim.check()

Checking model "gwf_advtidal"...

gwf_advtidal MODEL DATA VALIDATION SUMMARY:
  No errors or warnings encountered.

  Checks that passed:
    npf package: zero or negative horizontal hydraulic conductivity values
    npf package: vertical hydraulic conductivity values below checker threshold of 1e-11
    npf package: vertical hydraulic conductivity values above checker threshold of 100000.0
    npf package: horizontal hydraulic conductivity values below checker threshold of 1e-11
    npf package: horizontal hydraulic conductivity values above checker threshold of 100000.0
    ghb-tidal package: BC indices valid
    ghb-tidal package: not a number (Nan) entries
    ghb-tidal package: BC in inactive cells
    wel package: BC indices valid
    wel package: not a number (Nan) entries
    wel package: BC in inactive cells
    riv package: BC indices valid
    riv package: not a number (Nan) entries
    riv package: BC in inactive cells
    rch-zone_1 package: BC indices valid
    rch-zone_1

In [26]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims...
  writing model gwf_advtidal...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package sto...
    writing package ghb-tidal...
    writing package obs_0...
    writing package ghb_ts...
    writing package wel...
    writing package wel_ts...
    writing package riv...


    writing package riv_ts...
    writing package rch-zone_1...
    writing package rch_1_ts...
    writing package rch-zone_2...
    writing package rch_2_ts...
    writing package rch-zone_3...
    writing package rch_3_ts...
    writing package evt...
    writing package oc...
    writing package obs...


In [27]:
success, buff = sim.run_simulation()
assert success, "MODFLOW did not terminate normally!"

FloPy is using the following executable to run the model: ..\..\..\..\..\..\Software\mf6.4.1\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                        VERSION 6.4.1 Release 12/09/2022

   MODFLOW 6 compiled Dec 09 2022 18:18:37 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the US

In [28]:
sim.delete_output_files()